In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [2]:
#read in data
df = pd.read_csv('data/data.csv')
df = df.dropna()
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [3]:
#
x1 = df['oldpeak']
x2 = df['ca']
x3 = df['thal']
x4 = df['exang']
type(x1)

pandas.core.series.Series

In [4]:
X = np.array([x1, x2, x3, x4]).T
# X = np.array(df.thal)
# Y = np.array(df['num'])
Y = np.array(df['num'].apply(lambda x:1 if x>0 else 0))

In [5]:
# split the data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [6]:
X_train.shape

(267, 4)

In [7]:
Y_train.shape

(267,)

In [8]:
import random
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

class LogisticRegression(object):
    def __init__(self,lr=0.01, lam=0.1, fit_intercept=True):
        self.lr = lr
        self.lam = lam
        self.max_iter = 1e6
        self.tol = 1e-7
        self.fit_intercept = fit_intercept

    def _sigmoid(self,x):
        z = 1.0 / (1 + np.exp(-x))
        return z

    def _loss(self,X,y):
        """
        Penalized negative log likelihood of the targets under the current
        model.
            NLL = -1/N * (
                [sum_{i=0}^N y_i log(y_pred_i) + (1-y_i) log(1-y_pred_i)] -
                (gamma ||b||) / 2
            )
        """
        N, M = X.shape
        p = self._sigmoid(np.dot(X, self.w))
        c1 = y * np.log(p)
        c2 = (1 - y) * np.log(1-p)
        loss = (-sum(c1 + c2) + 0.5 * self.lam * sum(self.w ** 2)) / N
        return loss
        # loss = - np.log(p[y==1]).sum() - np.log(1- p[y==0]).sum()
        # penalty = 0.5 * self.lam * np.linalg.norm(self.w, ord=2)**2
        # return (loss + penalty) / N

    def fit(self, X, Y):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]   # add bias

        l_prev = np.inf
        self.w = np.random.randn(X.shape[1])

        for i in range(int(self.max_iter)):
            self.w -= self.lr * self._gradient(X,Y)
            loss = self._loss(X,Y)
            if l_prev - loss < self.tol:
                return
            l_prev = loss
            # if i % 100 == 0:
            #     print(i,'iteration, loss=',loss)

    def _gradient(self,X,y):
        y_pred = self._sigmoid(np.dot(X, self.w))
        g_l2norm = self.lam * self.w
        g = - (np.dot(y - y_pred, X) + g_l2norm) / X.shape[0]
        return g

    def predict(self,X):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]),X]
        y_pred = self._sigmoid(np.dot(X, self.w))
        np.putmask(y_pred, y_pred >= 0.5, 1.0)
        np.putmask(y_pred, y_pred < 0.5, 0.0)
        return y_pred

# Use feature 'oldpeak', 'ca', 'thal' and 'exang'

In [9]:
log_regression4 = LogisticRegression()
log_regression4.fit(X_train,Y_train)
log_regression4.w

array([-4.03543601,  0.53368121,  1.00009513,  0.45960527,  1.48556878])

In [10]:
y_pred4 = log_regression4.predict(X_test)
y_pred4.shape

(30,)

In [11]:
def calculate_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()

    return accuracy

In [14]:
accuracy4 = calculate_accuracy(y_pred4, Y_test)
accuracy4

0.8333333333333334

In [18]:
def calculate_precision(predictions, labels):
    true_positives = np.sum((predictions == 1) & (labels == 1))
    predicted_positives = np.sum(predictions == 1)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def calculate_recall(predictions, labels):
    true_positives = np.sum((predictions == 1) & (labels == 1))
    actual_positives = np.sum(labels == 1)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def calculate_confusion_matrix(predictions, labels):
    tp = np.sum((predictions == 1) & (labels == 1))
    tn = np.sum((predictions == 0) & (labels == 0))
    fp = np.sum((predictions == 1) & (labels == 0))
    fn = np.sum((predictions == 0) & (labels == 1))
    return np.array([[tp, fp], [fn, tn]])

# Now calculate each metric
precision4 = calculate_precision(y_pred4, Y_test)
recall4 = calculate_recall(y_pred4, Y_test)
f1_score4 = calculate_f1_score(precision4, recall4)
confusion_matrix4 = calculate_confusion_matrix(y_pred4, Y_test)

# Print the results
print(f"Accuracy: {accuracy4}")
print(f"Precision: {precision4}")
print(f"Recall: {recall4}")
print(f"F1 Score: {f1_score4}")
print("Confusion Matrix:")
print(confusion_matrix4)

Accuracy: 0.8333333333333334
Precision: 0.9090909090909091
Recall: 0.7142857142857143
F1 Score: 0.8
Confusion Matrix:
[[10  1]
 [ 4 15]]


# Use feature 'oldpeak', 'ca' and 'thal'

In [ ]:
log_regression3 = LogisticRegression()
log_regression3.fit(X_train,Y_train)
log_regression3.w

array([-4.03522675,  0.53342582,  1.0000852 ,  0.45946893,  1.48795649])

In [ ]:
y_pred3 = log_regression3.predict(X_test)
y_pred3.shape

(30,)

In [ ]:
def calculate_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()

    return accuracy

In [ ]:
accuracy4 = calculate_accuracy(y_pred3, Y_test)
accuracy4

0.8333333333333334

In [ ]:
def calculate_precision(predictions, labels):
    true_positives = np.sum((predictions == 1) & (labels == 1))
    predicted_positives = np.sum(predictions == 1)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def calculate_recall(predictions, labels):
    true_positives = np.sum((predictions == 1) & (labels == 1))
    actual_positives = np.sum(labels == 1)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def calculate_confusion_matrix(predictions, labels):
    tp = np.sum((predictions == 1) & (labels == 1))
    tn = np.sum((predictions == 0) & (labels == 0))
    fp = np.sum((predictions == 1) & (labels == 0))
    fn = np.sum((predictions == 0) & (labels == 1))
    return np.array([[tp, fp], [fn, tn]])

# Now calculate each metric
precision3 = calculate_precision(y_pred3, Y_test)
recall3 = calculate_recall(y_pred3, Y_test)
f1_score3 = calculate_f1_score(precision, recall)
confusion_matrix3 = calculate_confusion_matrix(y_pred3, Y_test)

# Print the results
print(f"Accuracy: {accuracy3}")
print(f"Precision: {precision3}")
print(f"Recall: {recall3}")
print(f"F1 Score: {f1_score3}")
print("Confusion Matrix:")
print(confusion_matrix3)


Accuracy: 0.8333333333333334
Precision: 0.9090909090909091
Recall: 0.7142857142857143
F1 Score: 0.8
Confusion Matrix:
[[10  1]
 [ 4 15]]


# Use feature 'oldpeak' and 'thal'

In [ ]:
log_regression = LogisticRegression()
log_regression.fit(X_train,Y_train)
log_regression.w

array([-4.03568611,  0.53386702,  1.00011855,  0.45971407,  1.48393749])

In [ ]:
Y_test.shape

(30,)

In [ ]:
y_pred = log_regression.predict(X_test)
y_pred.shape

(30,)

In [ ]:
def calculate_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()

    return accuracy

In [ ]:
accuracy = calculate_accuracy(y_pred, Y_test)
accuracy

0.8333333333333334

In [ ]:
def calculate_precision(predictions, labels):
    true_positives = np.sum((predictions == 1) & (labels == 1))
    predicted_positives = np.sum(predictions == 1)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def calculate_recall(predictions, labels):
    true_positives = np.sum((predictions == 1) & (labels == 1))
    actual_positives = np.sum(labels == 1)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def calculate_confusion_matrix(predictions, labels):
    tp = np.sum((predictions == 1) & (labels == 1))
    tn = np.sum((predictions == 0) & (labels == 0))
    fp = np.sum((predictions == 1) & (labels == 0))
    fn = np.sum((predictions == 0) & (labels == 1))
    return np.array([[tp, fp], [fn, tn]])

# Now calculate each metric
precision = calculate_precision(y_pred, Y_test)
recall = calculate_recall(y_pred, Y_test)
f1_score = calculate_f1_score(precision, recall)
confusion_matrix = calculate_confusion_matrix(y_pred, Y_test)

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")
print("Confusion Matrix:")
print(confusion_matrix)


Accuracy: 0.8333333333333334
Precision: 0.9090909090909091
Recall: 0.7142857142857143
F1 Score: 0.8
Confusion Matrix:
[[10  1]
 [ 4 15]]


In [ ]:
# # -*- coding: utf-8 -*-
# """logistic-regression-heart-disease.ipynb

# Automatically generated by Colaboratory.

# Original file is located at
#     https://colab.research.google.com/drive/1XHL3s2zZ2jRf4oSdYHL37sv5wmZIGTVw
# """

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import random

# df = pd.read_csv('/Users/yuehuasong/Desktop/2023-09-winter01-courses/518-machine-learninng/project-22/data.csv')
# df = df.dropna()
# df

# x1 = df['oldpeak']
# x2 = df['ca']
# x3 = df['thal']
# x4 = df['exang']
# type(x1)

# X = np.array([x1, x2, x3, x4]).T
# # X = np.array(df.thal)
# # Y = np.array(df['num'])
# Y = np.array(df['num'].apply(lambda x:1 if x>0 else 0))

# # split the data
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

# X_train.shape

# Y_train.shape

# import random
# import numpy as np
# import pandas as pd
# from sklearn.datasets import load_iris
# import matplotlib.pyplot as plt

# class LogisticRegression(object):
#     def __init__(self,lr=0.01, lam=0.1, fit_intercept=True):
#         self.lr = lr
#         self.lam = lam
#         self.max_iter = 1e6
#         self.tol = 1e-7
#         self.fit_intercept = fit_intercept

#     def _sigmoid(self,x):
#         z = 1.0 / (1 + np.exp(-x))
#         return z

#     def _loss(self,X,y):
#         """
#         Penalized negative log likelihood of the targets under the current
#         model.
#             NLL = -1/N * (
#                 [sum_{i=0}^N y_i log(y_pred_i) + (1-y_i) log(1-y_pred_i)] -
#                 (gamma ||b||) / 2
#             )
#         """
#         N, M = X.shape
#         p = self._sigmoid(np.dot(X, self.w))
#         c1 = y * np.log(p)
#         c2 = (1 - y) * np.log(1-p)
#         loss = (-sum(c1 + c2) + 0.5 * self.lam * sum(self.w ** 2)) / N
#         return loss
#         # loss = - np.log(p[y==1]).sum() - np.log(1- p[y==0]).sum()
#         # penalty = 0.5 * self.lam * np.linalg.norm(self.w, ord=2)**2
#         # return (loss + penalty) / N

#     def fit(self, X, Y):
#         if self.fit_intercept:
#             X = np.c_[np.ones(X.shape[0]), X]   # add bias

#         l_prev = np.inf
#         self.w = np.random.randn(X.shape[1])

#         for i in range(int(self.max_iter)):
#             self.w -= self.lr * self._gradient(X,Y)
#             loss = self._loss(X,Y)
#             if l_prev - loss < self.tol:
#                 return
#             l_prev = loss
#             # if i % 100 == 0:
#             #     print(i,'iteration, loss=',loss)

#     def _gradient(self,X,y):
#         y_pred = self._sigmoid(np.dot(X, self.w))
#         g_l2norm = self.lam * self.w
#         g = - (np.dot(y - y_pred, X) + g_l2norm) / X.shape[0]
#         return g

#     def predict(self,X):
#         if self.fit_intercept:
#             X = np.c_[np.ones(X.shape[0]),X]
#         y_pred = self._sigmoid(np.dot(X, self.w))
#         np.putmask(y_pred, y_pred >= 0.5, 1.0)
#         np.putmask(y_pred, y_pred < 0.5, 0.0)
#         return y_pred

# """# Use feature 'oldpeak', 'ca', 'thal' and 'exang'"""

# log_regression = LogisticRegression()
# log_regression.fit(X_train,Y_train)
# log_regression.w

# y_pred = log_regression.predict(X_test)
# y_pred.shape

# def calculate_accuracy(predictions, labels):
#     predictions_correct = predictions == labels
#     accuracy = predictions_correct.mean()

#     return accuracy

# accuracy = calculate_accuracy(y_pred, Y_test)
# accuracy